In [ ]:
!pip install langchain_openai pydantic

In [ ]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SequentialChain
from langchain.memory import SimpleMemory
from datetime import datetime

llm = OpenAI()

template_rapper: str = """Você é um compositor de rap renomado. Sua missão é criar uma letra de rap
inspirada no tema fornecido.

Tema da música:
{input}"""

prompt_template_rapper: PromptTemplate = PromptTemplate(
   input_variables=["input"], template=template_rapper)

cadeia_rapper: LLMChain = LLMChain(
   llm=llm, output_key="letra", prompt=prompt_template_rapper)

template_verificador: str = """Você é responsável por revisar letras de rap. Seu trabalho é verificar se as letras contêm
algum conteúdo violento ou linguagem inadequada.

Por favor, responda no formato de um dicionário Python:
letra: a letra recebida
Palavras_violentas: Verdadeiro ou Falso

Aqui está a letra a ser verificada:
{letra}"""

prompt_template_verificador: PromptTemplate = PromptTemplate(
   input_variables=["letra"], template=template_verificador)

cadeia_verificador: LLMChain = LLMChain(
   llm=llm, output_key="letra_verificada", prompt=prompt_template_verificador)

template_final: str = """Você é responsável pela verificação final das letras de rap. Seu trabalho é garantir que
a letra revisada esteja dentro dos padrões aceitáveis.

Sua resposta final deve ser no formato de dicionário Python:
letra: a letra recebida
Data e hora da verificação: {data_hora_verificacao}
Verificada por um humano: {verificada_por_humano}

Aqui está a letra revisada:
{letra_verificada}"""

# Criando o template de prompt para a revisão final
prompt_template_final: PromptTemplate = PromptTemplate(
   input_variables=["letra_verificada", "data_hora_verificacao", "verificada_por_humano"],
   template=template_final
)

cadeia_final: LLMChain = LLMChain(
   llm=llm, output_key="saida_final", prompt=prompt_template_final)

cadeia_sequencial: SequentialChain = SequentialChain(
   memory=SimpleMemory(memories={
                       "data_hora_verificacao": str(datetime.utcnow()), "verificada_por_humano": "Falso"}),
   chains=[cadeia_rapper, cadeia_verificador, cadeia_final],
   input_variables=["input"],
   output_variables=["saida_final"],
   verbose=True)

resultado = cadeia_sequencial.run(input="violência nas ruas")
print(resultado)

In [ ]:
from operator import itemgetter
from typing import Literal
from typing_extensions import TypedDict
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

prompt_acoes = ChatPromptTemplate.from_messages(
   [
       ("system", "Você é um especialista em investimentos em ações."),
       ("human", "{query}"),
   ]
)

prompt_renda_fixa = ChatPromptTemplate.from_messages(
   [
       ("system", "Você é um especialista em investimentos de renda fixa."),
       ("human", "{query}"),
   ]
)

chain_acoes = prompt_acoes | llm | StrOutputParser()
chain_renda_fixa = prompt_renda_fixa | llm | StrOutputParser()

route_system = "Roteie a pergunta do usuário para o especialista em ações ou renda fixa."
route_prompt = ChatPromptTemplate.from_messages(
   [
       ("system", route_system),
       ("human", "{query}"),
   ]
)

class RouteQuery(TypedDict):
   destination: Literal["acoes", "renda_fixa"]

route_chain = (
   route_prompt
   | llm.with_structured_output(RouteQuery)
   | itemgetter("destination")
)

chain = {
   "destination": route_chain,
   "query": lambda x: x["query"],
} | RunnableLambda(
   lambda x: chain_acoes if x["destination"] == "acoes" else chain_renda_fixa,
)

resultado = chain.invoke({"query": "Quais são os riscos de investir em ações de tecnologia?"})
print(resultado)